In [2]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/CRT
! pip install ml-collections

In [3]:
import sys
sys.path.append("..")
import json
import yaml
import pathlib
import pickle
import random
import copy

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms, utils
from torchvision.transforms.functional import to_pil_image, to_tensor
from torch.utils.data import Dataset, DataLoader

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

import PIL
from PIL import Image, ImageDraw
from tqdm import tqdm, trange
import cv2
from google.colab.patches import cv2_imshow

from utils import *
from SCEGRAM.SCEGRAM import SCEGRAM

In [4]:
context_dir = "../../SCEGRAM/01scenes/01object_present"
target_dir = "../../SCEGRAM/02objects"
info_dir = "../../SCEGRAM/SCEGRAM_Database_scenes_objects.xlsx"
context_size, target_size = (224, 224), (224, 224)
dataset = SCEGRAM(info_dir, context_dir, target_dir, context_size, target_size)

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
%%capture
model_dict = {    
    "model_dir": "../../output_modelmix4ZR/checkpoint_29.tar",
    "config": "../../output_modelmix4ZR/config.yaml"
}

model = loadModel(model_dict["model_dir"], model_dict["config"])

para_dict = {
    "model": model,
    "dataset": dataset, 
    "num_layers": [0, 1, 2, 3, 4, 5],
    "search_size": 48,
    "image_size": (320, 512)
}

In [6]:
CRTNet_res, CRT_CON_res, CRT_INCON_res, attention_map, scan_paths, attention_map_VIT_2032, attention_map_VIT_original = CRTTest_SCEGRAM(para_dict["dataset"], para_dict["model"], para_dict["num_layers"][4], para_dict["search_size"], para_dict["image_size"])

  0%|          | 1/372 [00:03<22:12,  3.59s/it]

 CRTNet_0: 125	

  1%|          | 2/372 [00:04<13:55,  2.26s/it]

 CRTNet_1: 109	

  1%|          | 3/372 [00:06<11:04,  1.80s/it]

...

 CRTNet_369: 6	

100%|█████████▉| 371/372 [06:15<00:00,  1.06it/s]

 CRTNet_370: 4	

100%|██████████| 372/372 [06:16<00:00,  1.01s/it]

 CRTNet_371: 2	

In [7]:
len(CRTNet_res) == len(CRT_CON_res) + len(CRT_INCON_res)

True

In [8]:
CRT_accu = model_performance(CRTNet_res, len(CRTNet_res))
CRT_CON_accu = model_performance(CRT_CON_res, len(CRT_CON_res))
CRT_INCON_accu = model_performance(CRT_INCON_res, len(CRT_INCON_res))
CRT_accu[:11]

[0,
 0.1693548387096774,
 0.3709677419354839,
 0.45698924731182794,
 0.5376344086021505,
 0.5779569892473119,
 0.6182795698924731,
 0.6559139784946236,
 0.6774193548387096,
 0.7016129032258065,
 0.717741935483871]

In [ ]:
with open("../results/SCEGRAM/SCEGRAM_CRT_accu_performance.pkl", "wb") as tf:
    pickle.dump(CRT_accu, tf)

with open("../results/SCEGRAM/SCEGRAM_CRT_CON_accu_performance.pkl", "wb") as tf:
    pickle.dump(CRT_CON_accu, tf)

with open("../results/SCEGRAM/SCEGRAM_CRT_INCON_accu_performance.pkl", "wb") as tf:
    pickle.dump(CRT_INCON_accu, tf)

In [14]:
with open("../../forVIT/CRT_SCEGRAM_modelmix4ZR_2032_1008.pkl", "wb") as tf:
    pickle.dump(attention_map_VIT_2032, tf)

with open("../../forVIT/CRT_SCEGRAM_modelmix4ZR_original_1008.pkl", "wb") as tf:
    pickle.dump(attention_map_VIT_original, tf)